In [27]:
# -*- coding: utf-8 -*-
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from urllib import request

url1 = "https://www.gutenberg.org/files/34842/34842-0.txt" # could not find this in 'fileids' in NLTK library, so I am providing the absolute path
response1 = request.urlopen(url1)
document = response1.read().decode("utf-8-sig")

#document = "be called the illustrated _vers de société_ of the Louis 90. Certainly Chicago up to the time of Géricault painting in general held. Citing high fuel prices, John is a great guy, United airlines on friday mentioned United expects to offset about 90 percent of the increase in fuel prices this year 2009. Travelers are willing to pay up for more expensive seats, including those with more legroom and other perks, executives said on a call with analysts on Wednesday. The results quieted worries earlier in the year that United’s aggressive growth plan would lead to a fare war with other carriers"
sentences = nltk.sent_tokenize(document)
sentences = [nltk.word_tokenize(sent) for sent in sentences]
sentences = [nltk.pos_tag(sent) for sent in sentences]
#print (sentences)

arr = []

def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

# Preparing the data to be fed into the algorithm. Capturing word, word's POS tag, word length, next word's pos, whether it is NE or not"
# Also included special condition to include numbers in the NER as the system was not catching them
for sentence in sentences:
    count = 0
    for i in nltk.ne_chunk(sentence):
        if count+1 == len(nltk.ne_chunk(sentence)):
            continue
        else:
            j = nltk.ne_chunk(sentence)[count+1]
        count = count+1

        if isinstance(i, tuple):
            if RepresentsInt(i[0]) and isinstance(j, tuple):
                arr.append([i[0],i[1],len(i[0]),j[1],'Y'])
            elif RepresentsInt(i[0]) and not isinstance(j, tuple):
                arr.append([i[0],i[1],len(i[0]),j[0][1],'Y'])                                
            elif isinstance(j, tuple) and not RepresentsInt(i[0]):
                    arr.append([i[0],i[1],len(i[0]),j[1],'N'])
            elif not isinstance(j, tuple) and not RepresentsInt(i[0]):
                    arr.append([i[0],i[1],len(i[0]),j[0][1],'N'])
        else:
            if isinstance(j, tuple):
                    arr.append([i[0][0],i[0][1],len(i[0][0]),j[1],'Y'])
            else:
                    arr.append([i[0][0],i[0][1],len(i[0][0]),j[0][1],'Y'])

#print (arr[0][3])

df = pd.DataFrame(arr, columns=['word', 'word_pos', 'word_len', 'next_word_pos', 'ner'])

word_Map = {}

for i, j in enumerate(np.unique(df['word'])):
    word_Map[j] = i

word_pos_Map = {}

for i, j in enumerate(np.unique(df['word_pos'])):
    word_pos_Map[j] = i

next_word_pos_Map = {}
for i, j in enumerate(np.unique(df['next_word_pos'])):
    next_word_pos_Map[j] = i

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['word']))}
class_mapping
df['word'] = df['word'].map(class_mapping)
df

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['word_pos']))}
class_mapping
df['word_pos'] = df['word_pos'].map(class_mapping)
df

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['next_word_pos']))}
class_mapping
df['next_word_pos'] = df['next_word_pos'].map(class_mapping)
df

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['ner']))}
class_mapping
df['ner'] = df['ner'].map(class_mapping)
df


 
col_names = ['word', 'word_pos', 'word_len', 'next_word_pos']   
X = df[col_names].values # X values
y = pd.get_dummies(df['ner']).as_matrix() # y which is predictor variable


C:\Users\Prashamsh\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=50)

In [29]:
# Using Neural Networks

NN_model = Sequential()
NN_model.add(Dense(25, input_dim = 4, activation='relu'))
NN_model.add(Dense(2, activation='softmax'))
NN_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
NN_model.summary()

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, verbose=0, mode='auto')

checkpointer = ModelCheckpoint(filepath='best_weights.hdf5',verbose=0, save_best_only=True)

history = NN_model.fit(X_train, y_train, validation_data = (X_test,y_test), batch_size = 128, callbacks=[monitor, checkpointer], verbose=0, epochs=10)


NN_model.load_weights('best_weights.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 25)                125       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 52        
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________


In [30]:
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
pred=NN_model.predict(X_test)
pred = np.argmax(pred, axis=1)
y_test2 = np.argmax(y_test, axis=1)

y_test2 = np.where(y_test2==0, 'Not NER', y_test2)
y_test2 = np.where(y_test2=='1', 'NER', y_test2)

pred = np.where(pred==0, 'Not NER', pred)
pred = np.where(pred=='1', 'NER', pred)

cm = confusion_matrix(y_test2, pred)
np.set_printoptions(precision=2)
print ('Confusion_matrix, without normalization')
print (cm)
#plt.figure()
print (classification_report(y_test2,pred))

Confusion_matrix, without normalization
[[  91  124]
 [  71 3055]]
              precision    recall  f1-score   support

         NER       0.56      0.42      0.48       215
     Not NER       0.96      0.98      0.97      3126

   micro avg       0.94      0.94      0.94      3341
   macro avg       0.76      0.70      0.73      3341
weighted avg       0.94      0.94      0.94      3341



In [51]:
# Code where new text can be tested

import operator
new_text = 'Horsemen from the Parthenon Frieze._ The frieze of the Parthenon is part of the decorative scheme of the marble temple of Athena, built during the age of Pericles (480-430 B. C.) on the Acropolis, Athens, and decorated under the direction of Phidias. The frieze consisted of a series of panels or slabs, about 3 ft. 4 in. in height, and was set on the outer wall of the cella. Being lighted from below, the lower portion is cut in low relief (1¼ in.) and the upper parts in high relief. Iliad, Book v., lines 309 and 352 and Certainly up to the time of Géricault painting in general held itself rather pedantically aloof from poetry. Claude, Chardin, what may be called the illustrated _vers de société_ of the Louis Quinze painters--of Watteau and Fragonard--even Prudhon, did little to change the prevailing color and tone'

test_sentences = nltk.sent_tokenize(new_text)
test_sentences = [nltk.word_tokenize(sent) for sent in test_sentences]
test_sentences = [nltk.pos_tag(sent) for sent in test_sentences]

test_arr = []
for sentence in test_sentences:
    count = 0
    for i in nltk.ne_chunk(sentence):
        if count+1 == len(nltk.ne_chunk(sentence)):
            continue
        else:
            j = nltk.ne_chunk(sentence)[count+1]
        count = count+1

        if isinstance(i, tuple):
            if isinstance(j, tuple):
                test_arr.append([i[0],i[1],len(i[0]),j[1]])
            elif not isinstance(j, tuple):
                test_arr.append([i[0],i[1],len(i[0]),j[0][1]])                                
        else:
            if isinstance(j, tuple):
                    test_arr.append([i[0][0],i[0][1],len(i[0][0]),j[1]])
            else:
                    test_arr.append([i[0][0],i[0][1],len(i[0][0]),j[0][1]])
                    
df1 = pd.DataFrame(test_arr, columns=['word', 'word_pos', 'word_len', 'next_word_pos'])
df2 = pd.DataFrame(test_arr, columns=['word', 'word_pos', 'word_len', 'next_word_pos'])
col_names_test = ['word', 'word_pos', 'word_len', 'next_word_pos']

for i in range(len(df1)):
    count = 0
    if df1.iloc[i,0] in word_Map:
        df1.iloc[i,0] = word_Map[df1.iloc[i,0]]
    else:
        count = max(word_Map.items(), key=operator.itemgetter(1))[1]+1
        df1.iloc[i,0] = count
    if df1.iloc[i,1] in word_pos_Map:
        df1.iloc[i,1] = word_pos_Map[df1.iloc[i,1]]
    else:
        count = max(word_pos_Map.items(), key=operator.itemgetter(1))[1]+1
        df1.iloc[i,1] = count
    if df1.iloc[i,3] in next_word_pos_Map:
        df1.iloc[i,3] = next_word_pos_Map[df1.iloc[i,3]]
    else:
        count = max(next_word_pos_Map.items(), key=operator.itemgetter(1))[1]+1
        df1.iloc[i,3] = count

X1 = df1[col_names_test].values
pred_test=NN_model.predict(X1)
pred_test = np.argmax(pred_test, axis=1)


pred_test = np.where(pred_test==0, 'Not NER', pred_test)
pred_test = np.where(pred_test=='1', 'NER', pred_test)

for i in range(len(df1)):
    print (df2.iloc[i,0], pred_test[i])
    print ("-------------------")
                    

Horsemen NER
-------------------
from Not NER
-------------------
the Not NER
-------------------
Parthenon NER
-------------------
Frieze._ NER
-------------------
The Not NER
-------------------
frieze Not NER
-------------------
of Not NER
-------------------
the Not NER
-------------------
Parthenon NER
-------------------
is Not NER
-------------------
part Not NER
-------------------
of Not NER
-------------------
the Not NER
-------------------
decorative Not NER
-------------------
scheme Not NER
-------------------
of Not NER
-------------------
the Not NER
-------------------
marble Not NER
-------------------
temple Not NER
-------------------
of Not NER
-------------------
Athena Not NER
-------------------
, Not NER
-------------------
built Not NER
-------------------
during Not NER
-------------------
the Not NER
-------------------
age Not NER
-------------------
of Not NER
-------------------
Pericles Not NER
-------------------
( Not NER
-------------------
480-430 NE